# Step 1. Preprocess and format JSON Data to NLP suitable formatting For LSTM

In [ ]:
#imports
import json
import os
import sys
import pickle
import re
import string
import numpy as np
import pandas as pd
from sklearn.metrics import  classification_report
from sklearn.metrics import  classification_report
import tensorflow as tf
from keras import initializers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow
from tensorflow.keras.layers import Embedding,Dropout,LSTM,Bidirectional,Concatenate,Input,Dense,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from IPython.display import Image
import numpy as np
from sklearn.metrics import  classification_report
import tensorflow as tf
import pickle
import pandas as pd


In [ ]:
# Function to load JSON data from a file
def load_json_file(file_path):
    with open(file_path, 'r') as json_file:
        return json.load(json_file)

# Function to create the "output" directory if it doesn't exist
def create_output_directory(output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

# Function to create emotion cause tags
def create_emotion_cause_tag(utterance, cause_span):
    return f"<{utterance['emotion']}>" + utterance['utterance'] + f"<cause>{cause_span}<\\cause> . <\\{utterance['emotion']}>"

# Function to preprocess JSON data and create output files
def preprocess_data_and_create_output(json_data, output_directory):
    create_output_directory(output_directory)

    for conv_id, conversation in json_data.items():
        with open(os.path.join(output_directory, f"{conv_id}.txt"), 'w') as output_file:
            for utterance in conversation[0]:
                if 'expanded emotion cause evidence' in utterance:
                    valid_evidence_indices = []
                    for evidence_idx_str, cause_span in zip(utterance['expanded emotion cause evidence'], utterance['expanded emotion cause span']):
                        try:
                            evidence_idx = int(evidence_idx_str) - 1
                            valid_evidence_indices.append(evidence_idx)
                        except ValueError:
                            pass
                    if valid_evidence_indices:
                        for evidence_idx in valid_evidence_indices:
                            output_line = create_emotion_cause_tag(conversation[0][evidence_idx], cause_span)
                            output_file.write(output_line + '\n')
                else:
                    output_line = f"<{utterance['emotion']}>" + utterance['utterance'] + f"<\\{utterance['emotion']}>"
                    output_file.write(output_line + '\n')

    print("Data preprocessing completed. Output files saved in the 'output' directory.")

# Function to merge unique lines from text files in a directory
def merge_unique_lines_from_directory(directory, merged_file):
    unique_lines = set()
    duplicate_count = 0

    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r') as file:
                lines = file.readlines()
                for line in lines:
                    line = line.strip()
                    if line not in unique_lines:
                        unique_lines.add(line)
                    else:
                        duplicate_count += 1

    with open(merged_file, 'w') as output_file:
        output_file.write('\n'.join(unique_lines))

    print(f"Merged {len(unique_lines)} unique lines.")
    if duplicate_count > 0:
        print(f"Found {duplicate_count} duplicate lines.")

In [ ]:
# Training Data
input_json_file = '/content/dailydialog_train.json'
output_directory = 'dailydialog_train'
merged_file_name = 'dailydialog_train_merged'
json_data = load_json_file(input_json_file)
preprocess_data_and_create_output(json_data, output_directory)
merge_unique_lines_from_directory(output_directory, f"{merged_file_name}.txt")


Data preprocessing completed. Output files saved in the 'output' directory.
Merged 8962 unique lines.
Found 1953 duplicate lines.


In [ ]:
# Testing Data
input_json_file = '/content/dailydialog_test.json'
output_directory = 'dailydialog_test'
merged_file_name = 'dailydialog_test_merged'
json_data = load_json_file(input_json_file)
preprocess_data_and_create_output(json_data, output_directory)
merge_unique_lines_from_directory(output_directory, f"{merged_file_name}.txt")


Data preprocessing completed. Output files saved in the 'output' directory.
Merged 2756 unique lines.
Found 444 duplicate lines.


In [ ]:
# Validation Data
input_json_file = '/content/dailydialog_valid.json'
output_directory = 'dailydialog_valid'
merged_file_name = 'dailydialog_valid_merged'
json_data = load_json_file(input_json_file)
preprocess_data_and_create_output(json_data, output_directory)
merge_unique_lines_from_directory(output_directory, f"{merged_file_name}.txt")


Data preprocessing completed. Output files saved in the 'output' directory.
Merged 599 unique lines.
Found 41 duplicate lines.


**Data Preparation**

In [ ]:
# Load train, test, and validation files
def load_data(file_path):
    documents = []
    with open(file_path, 'rt', encoding='latin') as doc_file:
        for line in doc_file:
            documents.append(line)
    return documents

train_documents = load_data('/content/dailydialog_train_merged.txt')
test_documents = load_data('/content/dailydialog_test_merged.txt')
validation_documents = load_data('/content/dailydialog_valid_merged.txt')


In [ ]:
# Define the set of emotion words set
words_set = {'embarrassed', 'horror', 'embarrassment', 'despair', 'excitement',
             'overjoyed', 'desolate', 'delighted', 'heartbreak', 'puzzlement',
             'infuriated', 'sorrow', 'startled', 'amusement', 'worried', 'astounded',
             'exhilarated', 'horrors', 'disquieted', 'gleeful', 'peeved', 'revulsion',
             'astonished', 'anguish', 'petrified', 'dejected', 'indignant', 'heartbroken',
             'furious', 'thrilled', 'despondency', 'horrified', 'annoyance', 'astonishment',
             'ashamed', 'happier', 'nonplussed', 'mad', 'anxious', 'sadness', 'exhilaration',
             'disgruntled', 'downcast', 'cross', 'crestfallen', 'agitated', 'devastated',
             'alarmed', 'disquiet', 'amused', 'irked', 'sad', 'livid', 'nervous', 'disgruntlement',
             'bewildered', 'down', 'depressed', 'dismayed', 'concerned', 'sickened', 'inconsolable',
             'flabbergasted', 'saddened', 'elation', 'anger', 'happy', 'frightened', 'revolted',
             'dismay', 'frustrated', 'fedup', 'angry', 'glum', 'ecstatic', 'griefstricken', 'delight',
             'miserable', 'exasperated', 'pleased', 'glee', 'bewilderment', 'grief', 'resentful',
             'excited', 'miffed', 'exasperation'}

# Define the filename for the emotion seeds pickle file
filename = 'emotion_seeds.pkl'

# Save the set to the pickle file
with open(filename, 'wb') as file:
    pickle.dump(words_set, file)

print(f'Saved the set to {filename}')


Saved the set to emotion_seeds.pkl


In [ ]:
# Emotion labels dictionary according to data
# {'anger',
#  'angry',
#  'disgust',
#  'excited',
#  'fear',
#  'happiness',
#  'happy',
#  'sad',
#  'sadness',
#  'surprise',
#  'surprised'}

#Extracts emotion-cause pairs from the given documents.
def get_emotion_cause_pairs(documents):

    with open('/content/emotion_seeds.pkl', 'rb') as f: #load saved emotion wordsets seeds
      emotion_seeds = pickle.load(f)

    document_clauses = []
    emotion_labels = []
    cause_labels = []

    emotion_count_dict = {}
    emotion_cause_pairs = []
    emotion_cause_pairs_label = []

    for i, line in enumerate(documents):
        #Determine the emotion of each statement
        emotion_of_line = ""
        if line.startswith("<happy>"):
          emotion_of_line = "happy"
        elif line.startswith("<happiness>"):
          emotion_of_line = "happy"
        elif line.startswith("<sad>"):
          emotion_of_line = "sad"
        elif line.startswith("<sadness>"):
          emotion_of_line = "sad"
        elif line.startswith("<surprise>"):
          emotion_of_line = "surprise"
        elif line.startswith("<surprised>"):
          emotion_of_line = "surprise"
        elif line.startswith("<disgust>"):
          emotion_of_line = "disgust"
        elif line.startswith("<anger>"):
          emotion_of_line = "angry"
        elif line.startswith("<angry>"):
          emotion_of_line = "angry"
        elif line.startswith("<shame>"):
          emotion_of_line = "shame"
        elif line.startswith("<fear>"):
          emotion_of_line = "fear"
        elif line.startswith("<excited>"):
          emotion_of_line = "excited"

        if emotion_count_dict.get(emotion_of_line) == None:
          emotion_count_dict[emotion_of_line] = 1
        else:
          emotion_count_dict[emotion_of_line] = emotion_count_dict[emotion_of_line]+1

        # get clauses from each of the document
        clauses = re.split("[.,!;:\"]+", line)
        emotion_clauses = []
        cause_clauses = []
        clauses_in_line = []
        for clause in clauses:

            cleaned_clause = remove_punctuation_from_clause(clause)

            clause_words = cleaned_clause.split()

            if len(clause_words) == 0:
                continue

            document_clauses.append(clause_words)
            clauses_in_line.append(clause_words)

            # Check for '<cause>' tag .
            if "<cause>" in clause:
                cause_labels.append(1)
                cause_clauses.append(clause_words)
            else:
                cause_labels.append(0)

            # Check if emotion seed is present in the clause.
            has_seed = False
            for word in clause_words:
                if word.lower() in emotion_seeds:
                    emotion_labels.append(1)
                    emotion_clauses.append(clause_words)
                    has_seed = True
                    break
            if not has_seed:
                emotion_labels.append(0)


        for m in range(len(clauses_in_line)):
          for n in range(len(clauses_in_line)):
              if clauses_in_line[m] in emotion_clauses and clauses_in_line[n] in cause_clauses:
                emotion_cause_pairs.append((clauses_in_line[m], clauses_in_line[n]))
                emotion_cause_pairs_label.append(emotion_of_line)
              elif m!=n:
                emotion_cause_pairs_label.append("None")
                emotion_cause_pairs.append((clauses_in_line[m], clauses_in_line[n]))


    return np.array(document_clauses), np.array(emotion_labels), np.array(cause_labels), emotion_cause_pairs,emotion_cause_pairs_label,emotion_count_dict

def remove_punctuation_from_clause(clause):
    clause = re.sub('<[^<]+>', "", clause)
    clause = clause.translate(str.maketrans('', '', string.punctuation))
    clause = clause.translate(str.maketrans('', '', string.digits))
    return clause


In [ ]:
document_clauses_train, emotion_labels_train, cause_labels_train, emotion_cause_pairs_train, emotion_cause_pairs_labels_train, emotion_count_dict_train = get_emotion_cause_pairs(train_documents)
document_clauses_test, emotion_labels_test, cause_labels_test, emotion_cause_pairs_test, emotion_cause_pairs_labels_test, emotion_count_dict_test = get_emotion_cause_pairs(test_documents)
document_clauses_validation, emotion_labels_validation, cause_labels_validation, emotion_cause_pairs_validation, emotion_cause_pairs_labels_validation, emotion_count_dict_validation = get_emotion_cause_pairs(validation_documents)

<ipython-input-8-d55afa1ed218>:107: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(document_clauses), np.array(emotion_labels), np.array(cause_labels), emotion_cause_pairs,emotion_cause_pairs_label,emotion_count_dict


In [ ]:
print("Train Data Statistics:")
print("Total number of documents in the train dataset: ", len(train_documents))
print("Total pairs of clauses in train data:", len(emotion_cause_pairs_train))
print("Total pairs of emotion-cause clauses in train data: ", len(emotion_cause_pairs_labels_train) - emotion_cause_pairs_labels_train.count("None"))


Train Data Statistics:
Total number of documents in the train dataset:  8962
Total pairs of clauses in train data: 90187
Total pairs of emotion-cause clauses in train data:  608


In [ ]:
print("\nTest Data Statistics:")
print("Total number of documents in the test dataset: ", len(test_documents))
print("Total pairs of clauses in test data:", len(emotion_cause_pairs_test))
print("Total pairs of emotion-cause clauses in test data: ", len(emotion_cause_pairs_labels_test) - emotion_cause_pairs_labels_test.count("None"))


Test Data Statistics:
Total number of documents in the test dataset:  2756
Total pairs of clauses in test data: 31065
Total pairs of emotion-cause clauses in test data:  148


In [ ]:
print("\nValidation Data Statistics:")
print("Total number of documents in the validation dataset: ", len(validation_documents))
print("Total pairs of clauses in validation data:", len(emotion_cause_pairs_validation))
print("Total pairs of emotion-cause clauses in validation data: ", len(emotion_cause_pairs_labels_validation) - emotion_cause_pairs_labels_validation.count("None"))



Validation Data Statistics:
Total number of documents in the validation dataset:  599
Total pairs of clauses in validation data: 6781
Total pairs of emotion-cause clauses in validation data:  30


In [ ]:
emotion_count_dict_train = dict()
for e in emotion_cause_pairs_labels_train:
  if emotion_count_dict_train.get(e) == None:
    emotion_count_dict_train[e] = 1
  else:
    emotion_count_dict_train[e]+=1
print("Emotion wise count in train data",emotion_count_dict_train)

emotion_count_dict_cv = dict()
for e in emotion_cause_pairs_labels_validation:
  if emotion_count_dict_cv.get(e) == None:
    emotion_count_dict_cv[e] = 1
  else:
    emotion_count_dict_cv[e]+=1
print("Emotion wise count in validation data",emotion_count_dict_cv)


emotion_count_dict_test = dict()
for e in emotion_cause_pairs_labels_test:
  if emotion_count_dict_test.get(e) == None:
    emotion_count_dict_test[e] = 1
  else:
    emotion_count_dict_test[e]+=1
print("Emotion wise count in test data",emotion_count_dict_test)

emotion_cause_pairs_train_labels = [ 0 if i == "None" else 1 for i in emotion_cause_pairs_labels_train]
emotion_cause_pairs_cv_labels = [ 0 if i == "None" else 1 for i in emotion_cause_pairs_labels_validation]
emotion_cause_pairs_test_labels  = [ 0 if i == "None" else 1 for i in emotion_cause_pairs_labels_test]

Emotion wise count in train data {'None': 89579, 'happy': 505, '': 34, 'disgust': 12, 'sad': 43, 'surprise': 3, 'angry': 9, 'fear': 2}
Emotion wise count in validation data {'None': 6751, '': 12, 'happy': 6, 'sad': 7, 'angry': 5}
Emotion wise count in test data {'None': 30917, '': 24, 'fear': 10, 'happy': 80, 'angry': 13, 'sad': 15, 'surprise': 6}


In [ ]:
len(emotion_cause_pairs_train_labels),len(emotion_cause_pairs_test_labels),len(emotion_cause_pairs_cv_labels)

(90187, 31065, 6781)

In [ ]:
######################################TRAIN DATA#################################################
clauses_train = []
emotion_train_labels = []
cause_train_labels = []

for i in range(len(emotion_cause_pairs_train)):
  if emotion_cause_pairs_train_labels[i] == 1:
    if emotion_cause_pairs_train[i][0] not in clauses_train:
        clauses_train.append(emotion_cause_pairs_train[i][0])
        emotion_train_labels.append(1)
        cause_train_labels.append(0)
    if emotion_cause_pairs_train[i][1] not in clauses_train:
        clauses_train.append(emotion_cause_pairs_train[i][1])
        emotion_train_labels.append(0)
        cause_train_labels.append(1)
#########################################VALIDATION DATA#################################################
clauses_cv = []
emotion_cv_labels = []
cause_cv_labels = []
for i in range(len(emotion_cause_pairs_validation)):
  if emotion_cause_pairs_cv_labels[i] == 1:
    if emotion_cause_pairs_validation[i][0] not in clauses_cv:
        clauses_cv.append(emotion_cause_pairs_validation[i][0])
        emotion_cv_labels.append(1)
        cause_cv_labels.append(0)
    if emotion_cause_pairs_validation[i][1] not in clauses_cv:
        clauses_cv.append(emotion_cause_pairs_validation[i][1])
        emotion_cv_labels.append(0)
        cause_cv_labels.append(1)

#########################################TEST DATA#################################################
clauses_test = []
emotion_test_labels = []
cause_test_labels = []
for i in range(len(emotion_cause_pairs_test)):
  if emotion_cause_pairs_test_labels[i] == 1:
    if emotion_cause_pairs_test[i][0] not in clauses_test:
        clauses_test.append(emotion_cause_pairs_test[i][0])
        emotion_test_labels.append(1)
        cause_test_labels.append(0)
    if emotion_cause_pairs_test[i][1] not in clauses_test:
        clauses_test.append(emotion_cause_pairs_test[i][1])
        emotion_test_labels.append(0)
        cause_test_labels.append(1)

In [ ]:
len(emotion_cause_pairs_train),emotion_cause_pairs_train_labels.count(1),len(emotion_cause_pairs_test),emotion_cause_pairs_test_labels.count(1),len(emotion_cause_pairs_validation),emotion_cause_pairs_labels_validation.count(0)

(90187, 608, 31065, 148, 6781, 0)

In [ ]:
len(clauses_train),len(clauses_cv),len(clauses_test)

(323, 17, 104)

#### **Prepare Embeddings**

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(document_clauses_train)

In [ ]:
print("Unique number of tokens in the data: ",len(tokenizer.word_index))

Unique number of tokens in the data:  5788


In [ ]:
#Maximum length of a clause(maximun number of words in a clause in the data)
MAX_SEQUENCE_LENGTH = 0

for clause in document_clauses_train:
  if len(clause) > MAX_SEQUENCE_LENGTH:
    MAX_SEQUENCE_LENGTH = len(clause)

MAX_SEQUENCE_LENGTH

34

In [ ]:
clauses_train_transformed = tokenizer.texts_to_sequences(clauses_train)
X_train = pad_sequences(clauses_train_transformed, maxlen=MAX_SEQUENCE_LENGTH,padding = 'post')

clauses_cv_transformed = tokenizer.texts_to_sequences(clauses_cv)
X_cv = pad_sequences(clauses_cv_transformed, maxlen=MAX_SEQUENCE_LENGTH,padding = 'post')

clauses_test_transformed = tokenizer.texts_to_sequences(clauses_test)
X_test = pad_sequences(clauses_test_transformed, maxlen=MAX_SEQUENCE_LENGTH,padding = 'post')

In [ ]:
#############################################################TRAIN DATA########################################################################################
emotion_from_emotion_cause_pairs_train,cause_from_emotion_cause_pairs_train = list(zip(*emotion_cause_pairs_train))
print("Number of Train Samples",len(emotion_from_emotion_cause_pairs_train),len(cause_from_emotion_cause_pairs_train),len(emotion_cause_pairs_train_labels))

emotion_from_emotion_cause_pairs_train_tansformed = tokenizer.texts_to_sequences(emotion_from_emotion_cause_pairs_train)
cause_from_emotion_cause_pairs_train_transformed = tokenizer.texts_to_sequences(cause_from_emotion_cause_pairs_train)

filter_model_emotion_train = pad_sequences(emotion_from_emotion_cause_pairs_train_tansformed, maxlen=MAX_SEQUENCE_LENGTH,padding = 'post')
filter_model_cause_train   = pad_sequences(cause_from_emotion_cause_pairs_train_transformed, maxlen=MAX_SEQUENCE_LENGTH,padding = 'post')
#############################################################VALIDATION DATA########################################################################################
emotion_from_emotion_cause_pairs_cv,cause_from_emotion_cause_pairs_cv = list(zip(*emotion_cause_pairs_validation))
print("Number of Validation Samples",len(emotion_from_emotion_cause_pairs_cv),len(cause_from_emotion_cause_pairs_cv),len(emotion_cause_pairs_cv_labels))

emotion_from_emotion_cause_pairs_cv_tansformed = tokenizer.texts_to_sequences(emotion_from_emotion_cause_pairs_cv)
cause_from_emotion_cause_pairs_cv_transformed = tokenizer.texts_to_sequences(cause_from_emotion_cause_pairs_cv)

filter_model_emotion_cv = pad_sequences(emotion_from_emotion_cause_pairs_cv_tansformed, maxlen=MAX_SEQUENCE_LENGTH,padding = 'post')
filter_model_cause_cv  = pad_sequences(cause_from_emotion_cause_pairs_cv_transformed, maxlen=MAX_SEQUENCE_LENGTH,padding = 'post')

Number of Train Samples 90187 90187 90187
Number of Validation Samples 6781 6781 6781


In [ ]:
import gensim.downloader as api
model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
EMBEDDING_DIM = 300

In [ ]:
word_index = tokenizer.word_index

In [ ]:

word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    try:
      embedding_vector = model.get_vector(word)
      embedding_matrix[i] = embedding_vector
    except:
      pass

In [ ]:
np.save('/content/emotion_embedding_layer_matrix.npy',embedding_matrix)

In [ ]:
embedding_matrix = np.load('/content/emotion_embedding_layer_matrix.npy')

In [ ]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.22558594, -0.01953125,  0.09082031, ...,  0.02819824,
        -0.17773438, -0.00604248],
       [ 0.20410156,  0.01318359,  0.07568359, ..., -0.21191406,
        -0.1328125 ,  0.10839844],
       ...,
       [-0.01672363, -0.11767578, -0.06445312, ..., -0.08789062,
        -0.03710938, -0.23535156],
       [ 0.23144531, -0.11816406,  0.19238281, ..., -0.11035156,
        -0.18261719,  0.11035156],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

### **EMOTION/ CAUSE CLASSIFICATION MODEL USING BILSTM**


In [ ]:
tf.keras.backend.clear_session()

BILSTM_HIDDEN_UNITS = 300

clause_input = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

#Using Word2Vec Model as pretrained embedding
embedding_layer = tf.keras.layers.Embedding(len(word_index) + 1,
                                            EMBEDDING_DIM,
                                            weights=[embedding_matrix],
                                            input_length=MAX_SEQUENCE_LENGTH,
                                            trainable=True,
                                            name='embedding_layer')

embedded_clause_input = embedding_layer(clause_input)
#Drop out for the embedding layer
embedded_clause_dropped_input = tf.keras.layers.Dropout(0.8,name='dropout_layer')(embedded_clause_input)
clause_bilstm_output = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(BILSTM_HIDDEN_UNITS
                                                    ))(embedded_clause_dropped_input)


output_emotion_1 = tf.keras.layers.Dense(units = 100, activation='sigmoid')(clause_bilstm_output)
output_emotion_2 = tf.keras.layers.Dense(units = 100, activation='sigmoid')(output_emotion_1)
output_emotion = tf.keras.layers.Dense(units = 1, activation='sigmoid', name = 'emotion')(output_emotion_2)

output_cause_1 = tf.keras.layers.Dense(units = 100, activation='sigmoid')(clause_bilstm_output)
output_cause_2 = tf.keras.layers.Dense(units = 100, activation='sigmoid')(output_cause_1)
output_cause = tf.keras.layers.Dense(units = 1,activation='sigmoid', name = 'cause')(output_cause_2)

model = tf.keras.models.Model(inputs=clause_input, outputs=[output_emotion, output_cause])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 34)]                 0         []                            
                                                                                                  
 embedding_layer (Embedding  (None, 34, 300)              1736700   ['input_1[0][0]']             
 )                                                                                                
                                                                                                  
 dropout_layer (Dropout)     (None, 34, 300)              0         ['embedding_layer[0][0]']     
                                                                                                  
 bidirectional (Bidirection  (None, 600)                  1442400   ['dropout_layer[0][0]']   

In [ ]:
model.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics = {'emotion': 'accuracy','cause': 'accuracy'}, loss_weights = [1, 1])

In [ ]:
model.fit(X_train, [np.array(emotion_train_labels), np.array(cause_train_labels)],
          validation_data = (X_cv, [np.array(emotion_cv_labels), np.array(cause_cv_labels)]),
          batch_size = 32,
          epochs = 10,
          verbose = True,
          shuffle = True)

Epoch 1/10
11/11 [==============================] - 7s 227ms/step - loss: 1.3958 - emotion_loss: 0.6955 - cause_loss: 0.7003 - emotion_accuracy: 0.5170 - cause_accuracy: 0.5139 - val_loss: 1.3688 - val_emotion_loss: 0.6805 - val_cause_loss: 0.6883 - val_emotion_accuracy: 0.5294 - val_cause_accuracy: 0.5294
Epoch 2/10
11/11 [==============================] - 1s 119ms/step - loss: 1.3228 - emotion_loss: 0.6588 - cause_loss: 0.6641 - emotion_accuracy: 0.6099 - cause_accuracy: 0.6099 - val_loss: 1.3679 - val_emotion_loss: 0.6835 - val_cause_loss: 0.6843 - val_emotion_accuracy: 0.5294 - val_cause_accuracy: 0.5294
Epoch 3/10
11/11 [==============================] - 2s 136ms/step - loss: 1.2679 - emotion_loss: 0.6316 - cause_loss: 0.6363 - emotion_accuracy: 0.6471 - cause_accuracy: 0.6502 - val_loss: 1.3369 - val_emotion_loss: 0.6670 - val_cause_loss: 0.6698 - val_emotion_accuracy: 0.5882 - val_cause_accuracy: 0.6471
Epoch 4/10
11/11 [==============================] - 1s 120ms/step - loss: 1.

In [ ]:
y_emotion_pred,y_cause_pred = model.predict(X_test)

4/4 [==============================] - 1s 5ms/step


In [ ]:
y_emotion_pred = [ 1 if i > 0.5 else 0 for i in y_emotion_pred ]
y_cause_pred = [ 1 if i > 0.5 else 0 for i in y_cause_pred ]

In [ ]:
class_report = classification_report(emotion_test_labels, y_emotion_pred, target_names=[0,1], output_dict=True)
report = pd.DataFrame(data = class_report)
report

,0,1,accuracy,macro avg,weighted avg
precision,0.936170,0.771930,0.846154,0.854050,0.861946
recall,0.771930,0.936170,0.846154,0.854050,0.846154
f1-score,0.846154,0.846154,0.846154,0.846154,0.846154
support,57.000000,47.000000,0.846154,104.000000,104.000000


In [ ]:
class_report = classification_report(cause_test_labels, y_cause_pred, target_names=[0,1], output_dict=True)
report = pd.DataFrame(data = class_report)
report

,0,1,accuracy,macro avg,weighted avg
precision,0.771930,0.936170,0.846154,0.854050,0.861946
recall,0.936170,0.771930,0.846154,0.854050,0.846154
f1-score,0.846154,0.846154,0.846154,0.846154,0.846154
support,47.000000,57.000000,0.846154,104.000000,104.000000
